In [2]:
import pandas as pd
import  numpy as np

train_df = pd.read_csv("Funding Successful Projects prediction/raw data/train.csv",
                       usecols=["final_status","keywords"],nrows =4000)
train_df.fillna(value="not known",inplace=True)
#train_df['all_words'] = train_df[["name","desc"]].apply(lambda x: ' '.join(x), axis=1)
target_train_df = train_df["final_status"]

In [8]:
train_df.head()

,keywords,final_status
0,drawing-for-dollars,1
1,sponsor-dereck-blackburn-lostwars-artist-in-re...,0
2,mr-squiggles,0
3,help-me-write-my-second-novel,1
4,support-casting-my-sculpture-in-bronze,0


In [5]:
def split_keywords(df):
    return df["keywords"].str.split(pat="-", n=-1, expand=False)
train_df["splitted_keywords"] = split_keywords(train_df)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
tfidf_train = tv.fit_transform(train_df["keywords"])

In [22]:
len(tv.get_feature_names()),tv.get_feature_names()

(7302,
 ['001',
  '00am',
  '01',
  '08',
  '10',
  '100',
  '1000',
  '101',
  '101s',
  '104',
  '10th',
  '11',
  '12',
  '12inch',
  '13',
  '13th',
  '14',
  '140',
  '14000',
  '144',
  '15',
  '16',
  '16mm',
  '16th',
  '1901',
  '1930s',
  '1950s',
  '1957',
  '196',
  '1968',
  '1970s',
  '1976',
  '1978',
  '1980',
  '1985',
  '1988',
  '1989',
  '1990',
  '1995',
  '1999',
  '1st',
  '20',
  '200',
  '2007',
  '2009',
  '201',
  '2010',
  '2010s',
  '2011',
  '2012',
  '20s',
  '21',
  '21st',
  '22',
  '227',
  '24',
  '25',
  '2600',
  '27',
  '2707',
  '28',
  '2d',
  '2e',
  '2morrow',
  '2nd',
  '30',
  '3000',
  '320',
  '350',
  '35mm',
  '36',
  '360',
  '365',
  '37',
  '38',
  '3d',
  '3rd',
  '40',
  '4000',
  '40love',
  '42',
  '469',
  '48',
  '4e',
  '4th',
  '50',
  '500',
  '500000',
  '504',
  '53',
  '55',
  '5th',
  '60',
  '6000',
  '60s',
  '63',
  '67',
  '6th',
  '731',
  '80s',
  '82',
  '90',
  '9000',
  '90s',
  '911',
  '99ents',
  '9th',
  'aaka

In [4]:
# Utility function to report best scores
from time import gmtime, strftime
def report(results, n_top=10):
    """
    :param results: list
    :param n_top: int
    :return: None
    """
    print("Report generated at {}".format(strftime("%d-%m-%Y %H:%M:%S")))
    print("")
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.6f} (std: {1:.6f}),time:{2:.3f}".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate],
                results["mean_fit_time"][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [23]:
#using the pipeline
import numpy as np
from time import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression,RidgeClassifier,SGDClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.neural_network import MLPClassifier

parameters = {}
#parameters = {'min_samples_split':[30,60,90], 
#              'n_estimators':[90],
#              "learning_rate":[0.001],
#              "loss":["deviance"]
#              }
#clf = GradientBoostingClassifier(n_estimators=60)
#clf = XGBClassifier()
#clf = SVC(kernel='poly', probability=True,C = 160,degree=3)
#clf = MLPClassifier( hidden_layer_sizes=(5,2))
#clf = MultinomialNB(alpha=0.01)
#clf = BernoulliNB()
#clf = RandomForestClassifier(n_estimators=30)
#parameters={"fit_intercept":[True,False]}
clf = LogisticRegression()

grid = GridSearchCV(clf,parameters,scoring='accuracy',cv=5)


t0 = time()
grid.fit(tfidf_train,train_df["final_status"])
print("done in %0.3fs" % (time() - t0)) 
report(grid.cv_results_,n_top=10)

C:\Users\user\Anaconda3.6\envs\python35env\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


done in 0.288s
Report generated at 24-06-2017 01:02:15

Model with rank: 1
Mean validation score: 0.641250 (std: 0.002372),time:0.043
Parameters: {}



We do some pipelining

In [14]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
models.append(('GBC', GradientBoostingClassifier(n_estimators=50)))
models.append(('RFC', RandomForestClassifier(n_estimators=50)))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
	kfold = model_selection.KFold(n_splits=6, random_state=seed)
	cv_results = model_selection.cross_val_score(model,tfidf_train.toarray(),train_df["final_status"], cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

GBC: 0.640283 (0.056943)


RFC: 0.642846 (0.069987)


SVM: 0.637909 (0.087969)


In [27]:
from time import time
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
#from xgboost import XGBClassifier


# build a classifier
clf = RandomForestClassifier()

# specify parameters and distributions to sample from
param_dist = {#"learning_rate":sp_uniform(0.06,0.1),
              "n_estimators":sp_randint(40,110),
               #'subsample': np.random.uniform(0.3,0.999,1000),
              'min_samples_split':sp_randint(30,150),
               }

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,scoring='accuracy')

start = time()
random_search.fit(tfidf_train.toarray(),train_df["final_status"])
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

report(random_search.cv_results_,n_top=10)

KeyboardInterrupt: 

In [10]:
report(random_search.cv_results_)

Report generated at 20-06-2017 19:07:01

Model with rank: 1
Mean validation score: 0.422921 (std: 0.183223),time:12.973
Parameters: {'learning_rate': 0.073852187138425771, 'min_samples_split': 267, 'n_estimators': 65}

Model with rank: 2
Mean validation score: 0.422838 (std: 0.183293),time:7.517
Parameters: {'learning_rate': 0.12001840513311865, 'min_samples_split': 61, 'n_estimators': 61}

Model with rank: 3
Mean validation score: 0.422828 (std: 0.183301),time:26.520
Parameters: {'learning_rate': 0.10597065524294731, 'min_samples_split': 43, 'n_estimators': 105}



#now do the actual testing

In [58]:
import pandas as pd
import  numpy as np
#from sklearn.feature_extraction.text import TfidfVectorizer

test_df = pd.read_csv("Funding Successful Projects prediction/raw data/test.csv",
                       usecols=["name","desc","project_id"])
test_df.fillna(value="not known",inplace=True)
test_df.drop(["name","desc"],axis=1,inplace=True)

test_text =preprocess_text(test_df)
tfidf_test = tv.transform(test_text)

In [60]:
#can skip straight to here if test data is loaded and clf is trained
submit_df = pd.DataFrame()
submit_df["project_id"] = test_df["project_id"]
submit_df["final_status"]=grid.predict(tfidf_test)

from time import strftime
current_time = strftime("%d-%b-%Y %H-%M-%S")
submit_df.to_csv("Funding Successful Projects prediction/output_data/successful projects detection"
                       "_results_{0}.csv".format(current_time)
                       ,index=False)